In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Bidirectional, LSTM, SimpleRNN,GRU,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
# Load the dataset
# df = pd.read_excel('../../dataset/Dataset_IoV.xlsx')  
df = pd.read_csv('..//..//dataset//preliminary_dataset.csv')  


# Separating features and target
X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

X_scaled = X

# Reshape input to be 3D [samples, timesteps, features] for LSTM
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [3]:
print(X_train.shape, y_train.shape,X_scaled.shape)

(80000, 1, 6) (80000,) (100000, 1, 6)


# Model

In [4]:
def build_drnn():
    model = Sequential()
    
    # Input layer
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    
    # DRNN layers with 3 hidden layers
    model.add(SimpleRNN(units=100,activation='relu',  return_sequences=True))
    model.add(SimpleRNN(units=100,activation='relu',  return_sequences=True))
    model.add(SimpleRNN(units=100, ))
    
    # Output layer
    model.add(Dense(units=1, activation='sigmoid'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])
    
    return model


In [5]:
# Instantiate and train the model
model = build_drnn()
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.2233 - mae: 0.1660 - mse: 0.0711 - val_loss: 0.0028 - val_mae: 0.0026 - val_mse: 2.3051e-04
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0023 - mae: 0.0014 - mse: 4.0749e-04 - val_loss: 3.8424e-04 - val_mae: 3.5079e-04 - val_mse: 3.3826e-05
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 9.0697e-04 - mae: 6.8101e-04 - mse: 1.7268e-04 - val_loss: 0.0037 - val_mae: 0.0032 - val_mse: 7.6847e-04
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 9.6370e-04 - mae: 6.0794e-04 - mse: 1.9846e-04 - val_loss: 1.7029e-04 - val_mae: 1.0150e-04 - val_mse: 4.0302e-05
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0013 - mae: 6.5073e-04 - mse: 3.5716e-04 - val_loss: 3.8414e-04 - val_mae: 2.9397e-04 - val_mse: 6.8347e-05
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 6.2741e-04 - mae: 2.9973e-04 - mse: 1.2485e-04 - val_loss: 3.5442e-05 - val_mae: 3.5149e-05 - val_mse: 5.5893

In [6]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Make predictions
y_pred = model.predict(X_test)




625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [7]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error (MSE): {mse:.16f}')

# Calculate RMSE
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse:.16f}')

Mean Squared Error (MSE): 0.0000329250397044
Root Mean Squared Error (RMSE): 0.0057380344809371


In [8]:
# import matplotlib.pyplot as plt

# # Visualizing the loss, MSE, and MAE
# plt.figure(figsize=(15, 5))

# # Loss plot
# plt.subplot(1, 3, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Model Loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MSE plot
# plt.subplot(1, 3, 2)
# plt.plot(history.history['mse'], label='Train MSE')
# plt.plot(history.history['val_mse'], label='Validation MSE')
# plt.title('Mean Squared Error')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# # MAE plot
# plt.subplot(1, 3, 3)
# plt.plot(history.history['mae'], label='Train MAE')
# plt.plot(history.history['val_mae'], label='Validation MAE')
# plt.title('Mean Absolute Error')
# plt.ylabel('MAE')
# plt.xlabel('Epoch')
# plt.legend(loc='upper right')

# plt.tight_layout()
# plt.show()


In [9]:
import time

# Start time
start_time = time.time()

# Make predictions
y_pred = model.predict(X_test)

# End time
end_time = time.time()

# Total AES time
total_time = end_time - start_time  

# AES per sample
aes_per_sample = (total_time / len(X_test)) * 1e6  # Convert to microseconds (µs)

print(f'Total AES Time: {total_time:.6f} seconds')
print(f'AES Time per Sample: {aes_per_sample:.2f} µs/sample')


625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Total AES Time: 1.720999 seconds
AES Time per Sample: 86.05 µs/sample
